In [1]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')


### Dataset

download kaggle dataset: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/download?datasetVersionNumber=1  
download liar dataset: https://www.cs.ucsb.edu/~william/data/liar_dataset.zip

In [2]:
from load_dataset import kaggle_dataset
from load_dataset import liar_dataset
kaggle_path = "./dataset/Fake-News/"
liar_path = "./dataset/Liar/"

# Pick dataset you want:
# If kaggle:
# dataset = kaggle_dataset(kaggle_path)
# If Liar:
dataset = liar_dataset(liar_path)

train_sentences = dataset.train_sentences
val_sentences = dataset.val_sentences
train_labels = dataset.train_labels
val_labels = dataset.val_labels

### Model

##### Word Enbedding:  
pre-trained text embedding model from TensorFlow Hub  
more text-embedding download in https://tfhub.dev/s?module-type=text-embedding  
NNLM:  
plain text -> value [batch_size, 128]  
BERT:  
plain text -> pooled_output [batch_size, 128]  
              sequence_output [batch_size, seq_length, 128]  

##### Network

In [3]:
# Split a small batch of dataset to continue (save time...)
i = 2000
train_x = train_sentences[:i]
val_x = val_sentences[:int(0.2*i)]
train_y = train_labels[:i]
val_y = val_labels[:int(0.2*i)]

In [4]:
cov_filters = 5
cov_kernel = 64
pool_size = 4
LSTM_units = 100
embedding = "bert"

# Select a model
from load_model import lstm
# lstm(LSTM_units, embedding)
from load_model import cnn_lstm
# cnn_lstm(cov_filters, cov_kernel, pool_size, LSTM_units, embedding)
from load_model import dense
# dense(embedding, neural=64)
from load_model import dense_dropout
# dense_dropout(embedding, dropout=0.1, neural_1=256, neural_2=64)

model = lstm(LSTM_units,embedding=embedding)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_history = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs=5)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['input_1[0][0]']                
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

### Attack (TBC)

In [5]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper
from textattack import AttackArgs
from textattack import Attacker
from textattack.datasets import Dataset

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        final_preds = torch.stack((1-logits, logits), dim=1)
        return final_preds

In [7]:
from textattack.attack_recipes import DeepWordBugGao2018 as attack_recipe

i = 100
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=5,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
# attacker.attack_dataset()

textattack: Unknown if model of class <class 'load_model.lstm'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
